## 컨트랙 개발

In [1]:
%%writefile BankV3.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0; //^0.5.0;

contract BankV3 {
    address owner;
    uint balance;
    uint256 timeToDeposit;
    
    event PrintLog(string);
    event Sent(address from, address to, uint amount );
    constructor() { //constructor() public {
        owner = msg.sender;
        balance = 0;
    }
    fallback() external { // v0.5.0 function() external {
        emit PrintLog("Fallback called");
    }
    function forwardTo(address payable _receiver) public payable onlyOwner {
        //require(msg.sender == owner);
        _receiver.transfer(msg.value);
        emit Sent(msg.sender, _receiver, msg.value);
    }
    function getBalance() public view returns(uint, uint) {
        return (balance, address(this).balance);
    }
    function deposit(uint amount) public payable onlyAfter {
        timeToDeposit = block.timestamp + 10 seconds; //timeToDeposit = now + 10 seconds;
        require(msg.value == amount);
        balance += amount;
    }
    function withdrawAll() public onlyOwner minBalance {
        balance -= address(this).balance;
        //require(msg.sender == owner);
        //v0.5.0 msg.sender.transfer(address(this).balance);
        payable(msg.sender).transfer(address(this).balance); 
    }
    modifier onlyOwner {
        require(msg.sender == owner);
        _;
    }
    modifier onlyAfter {
        require(block.timestamp >= timeToDeposit); //require(now >= timeToDeposit);
        _;
    }
    modifier minBalance {
        require(address(this).balance>101 wei);
        _;
    }
}

Writing BankV3.sol


## 컴파일

In [3]:
!solc-windows.exe --optimize --combined-json abi,bin BankV3.sol > BankV3.json

## 배포

In [4]:
%%writefile BankV3DeployFromFile.js
var Web3 = require('web3');
var _abiBinJson = require('./BankV3.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); // reading ['src/BankV3.sol:BankV3']
console.log("- contract name: ", contractName);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!! //SyntaxError: Unexpected token o in JSON at position 1
_bin=_abiBinJson.contracts[contractName].bin;

//console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 259210}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Writing BankV3DeployFromFile.js


In [6]:
!node BankV3DeployFromFile.js

- contract name:  [ 'BankV3.sol:BankV3' ]
Deploying the contract from 0xe41D6E1ca60ea1150976B7bF0F670b224F4B82D6
hash: 0x83c18622a25f1fb3d3faa759fda0c4ff128584e06bfeb724a0e61833a30767ea
---> The contract deployed to: 0xcF591E42C3184B8cbC3d76560b2Ae9B65F5CaBAe


## 사용
프로그램에 설정한 제약조건을 지켜서 실행헤야 한다.

10초 이내 저축
잔고 101보다 적은데 출금
블록체인에 send()가 필요한 함수는 비동기적으로 처리하기 위해 await로 처리한다. 비동기적으로 처리하면, 예를 들어 입금 deposit()하고 getBalance()하면 잔고에 입금분만큼 반영이 되어있는 것을 알 수 있다

In [7]:
%%writefile BankV3Use.js
var Web3=require('web3');
var _abiBinJson = require('./BankV3.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/BankV3.sol:BankV3']
//console.log("- contract name: ", contractName); //or console.log(contractName[0]);
_abiArray=_abiBinJson.contracts[contractName].abi; //use just as read from file
//_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!!
//_bin=_abiBinJson.contracts[contractName].bin;
console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);
var bank = new web3.eth.Contract(_abiArray,"0xcF591E42C3184B8cbC3d76560b2Ae9B65F5CaBAe");
//var filter = bank.PrintLog(function (error, result) {
//  if (!error)
//    console.log(result);
//});
//console.log(bank.sendTo(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, 100,{from:web3.eth.accounts[0],gas:100000}));

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    bank.methods.getBalance().call().then(console.log);
    await bank.methods.deposit(111).send({from: accounts[0], value:111});
    bank.methods.getBalance().call().then(console.log);
    await bank.methods.withdrawAll().send({from: accounts[0]});    //greater than 101
    bank.methods.getBalance().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));    
}
doIt()

Writing BankV3Use.js


In [9]:
!node BankV3Use.js

- ABI: [object Object],[object Object],[object Object],[object Object],[object Object],[object Object],[object Object],[object Object]
Account: 0xe41D6E1ca60ea1150976B7bF0F670b224F4B82D6
Balance before: 999997673235726389846
Result { '0': '0', '1': '0' }
Result { '0': '111', '1': '111' }
Balance after: 999997404032462255192
Balance diff: 269203264110592
Result { '0': '0', '1': '0' }


결과는 비동기 함수를 일시에 실행하기 때문에 바르게 출력되지 않고 있다. node창에서 실행을 해보면 알 수 있다.

deposit(100);
withdraw() ---> 실패 (프로그램에 101 wei 이상 제약조건)
deposit(111); value를 반드시 함수인자와 동일하게 입력해야 한다. REMIX에서 할 경우에도 마찬가지이다. Run 탭 상단의 value, Deployed Contracts 함수의 인자 두 필드에 동일한 금액을 넣어준다.
빠르게 (프로그램에 설정해 놓은 10초 이내) deposit(111) ---> 실패
queryBalance() --=> 211
deposit(111); ---> 10초가 지났으면 실행. 이때 마이닝을 해서 동기화를 시켜줄 필요가 있다.
queryBalance() ---> 322


 bank.methods.getBalance().call().then(console.log);              잔고 0
 0
 
 bank.methods.deposit(100).send({from:"0x4D2fF...", value:100});    입금 100
 
 //miner.start(1);admin.sleepBlocks(1);miner.stop();                마이닝
 
 bank.methods.getBalance().call().then(console.log);              입금금액 조회 100. 앞 입금거래를 마이닝하고 금액 증가.
 100                                                                
 
 bank.methods.deposit(100).send({from:"0x4D2fF...", value:111});    입금 'value'와 '인자'가 서로 다르면 입금 실패
 
 //miner.start(1);admin.sleepBlocks(1);miner.stop();                마이닝
 undefined
 
 bank.methods.getBalance().call().then(console.log);              실패하였으므로 잔고는 계속 100
 100
 
 bank.methods.deposit(111).send({from:"0x4D2fF...", value:111});    입금 111
 
 //miner.start(1);admin.sleepBlocks(1);miner.stop();                마이닝
 
 bank.methods.getBalance().call().then(console.log);              입금 100+111 = 211
 211
 
 bank.methods.deposit(111).send({from:"0x4D2fF...", value:111});    (10초 지나서) 입금 111, 마이닝하면 금액 증가함
 
 //miner.start(1);admin.sleepBlocks(1);miner.stop();
 
 bank.methods.getBalance().call().then(console.log);              입금 100+111+111 = 322
 322
 
 bank.methods.deposit(111).send({from:"0x4D2fF...", value:111});    입금 111, 마이닝하면 금액 증가함
 
 bank.methods.deposit(111).send({from:"0x4D2fF...", value:111});    입금 111, 10초 이내이므로 마이닝해도 금액 증가하지 않음
 //miner.start(1);admin.sleepBlocks(1);miner.stop();
 
 bank.methods.getBalance().call().then(console.log);              여러번 입금해도 10초 이내 거래는 실패하고 잔고가 늘지 않음
 433